# Zillow Data - Modelo Dimensional (Star Schema)
Este notebook cria as tabelas de dimensão e fato para análise de indicadores do mercado imobiliário Zillow.

In [ ]:
import duckdb
import pandas as pd
from datetime import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

## Conectar ao DuckDB e Carregar Dados de Origem

In [ ]:
conn = duckdb.connect('zillow_dwh.duckdb')

conn.execute('DROP SCHEMA IF EXISTS zillow_dwh CASCADE;')

## Criar Tabelas de Dimensão e Fato

### 1. Dimension Time

In [ ]:
conn.execute("""
DROP TABLE IF EXISTS dim_time
""")

conn.execute("""
CREATE TABLE dim_time (
    date_key DATE PRIMARY KEY,
    day_name VARCHAR,
    week_number INTEGER,
    month_number INTEGER,
    month_name VARCHAR,
)
""")

print("Dim Time table created successfully.")

### 2. Dimension Regions

In [ ]:
conn.execute("DROP TABLE IF EXISTS dim_country")
conn.execute("""
CREATE TABLE dim_country (
    country_key INTEGER PRIMARY KEY,
    country_name VARCHAR,
    country_code VARCHAR
)
""")
print("✓ dim_country created successfully.")


conn.execute("DROP TABLE IF EXISTS dim_region")
conn.execute("""
CREATE TABLE dim_region (
    region_key INTEGER PRIMARY KEY,
    country_key VARCHAR,
    region_type VARCHAR,
    zip_key INTEGER,
    neighborhood_name VARCHAR,
    city_name VARCHAR,
    county_name VARCHAR,
    metro_name VARCHAR,
    state_name VARCHAR,
    state_code VARCHAR,
    FOREIGN KEY (country_key) REFERENCES dim_country(country_key)
)
""")
print("✓ dim_region created successfully.")

### 3. Dimension Indicator

In [ ]:
conn.execute("""
DROP TABLE IF EXISTS dim_realestate_indicator
""")

conn.execute("""
CREATE TABLE dim_realestate_indicator (
    realestate_indicator_key VARCHAR PRIMARY KEY,
    indicator_name VARCHAR,
    indicator_description TEXT
)
""")

print("✓ dim_realestate_indicator table created successfully.")

### 4. Dimension Value

In [ ]:
conn.execute("""
DROP TABLE IF EXISTS dim_value
""")

conn.execute("""
CREATE TABLE dim_value (
    value_key INTEGER PRIMARY KEY,
    value DECIMAL,
    unit ENUM('USD_CURRENT', 'PERCENTAGE', 'DAYS', 'UNITS')
)
""")

print("✓ dim_value table created successfully.")

### 5. Dimension asset

In [ ]:
conn.execute("""
DROP TABLE IF EXISTS dim_asset
""")

conn.execute("""
CREATE TABLE dim_asset (
    asset_key VARCHAR PRIMARY KEY,
    ticker VARCHAR,
    type ENUM('CRYPTOCURRENCY', 'STOCK', 'ORE')
)
""")

print("✓ dim_asset table created successfully.")

### 6. Dimension Socioeconomical Indicator

In [ ]:
conn.execute("""
DROP TABLE IF EXISTS dim_socioeconomical_indicator
""")

conn.execute("""
CREATE TABLE dim_socioeconomical_indicator (
    socioeconomical_indicator_key VARCHAR PRIMARY KEY,
    name VARCHAR,
    description TEXT,
)
""")

print("✓ dim_socioeconomical_indicator table created successfully.")

### 7. Dimension CryptoStock Value

In [ ]:
conn.execute("""
DROP TABLE IF EXISTS dim_cryptostock_value
""")

conn.execute("""
CREATE TABLE dim_cryptostock_value (
    cryptostock_value_key INTEGER PRIMARY KEY,
    type ENUM('CRYPTO', 'STOCK'),
    high DECIMAL,
    low DECIMAL,
    volume DECIMAL,
    close DECIMAL
)
""")

print("✓ dim_cryptostock_value table created successfully.")

### 8. Fact Table

In [ ]:
conn.execute("""
DROP TABLE IF EXISTS fact_value
""")

conn.execute("""
CREATE TABLE fact_value (
    value_key INTEGER,
    socioeconomical_indicator_key VARCHAR,
    realestate_indicator_key VARCHAR,
    cryptostock_value_key INTEGER,
    country_key INTEGER,
    region_key INTEGER,
    date_key DATE,
    asset_key INTEGER,
    FOREIGN KEY (value_key) REFERENCES dim_value(value_key),
    FOREIGN KEY (asset_key) REFERENCES dim_asset(asset_key),
    FOREIGN KEY (socioeconomical_indicator_key) REFERENCES dim_socioeconomical_indicator(socioeconomical_indicator_key),
    FOREIGN KEY (realestate_indicator_key) REFERENCES dim_realestate_indicator(realestate_indicator_key),
    FOREIGN KEY (cryptostock_value_key) REFERENCES dim_cryptostock_value(cryptostock_value_key),
    FOREIGN KEY (country_key) REFERENCES dim_country(country_key),
    FOREIGN KEY (region_key) REFERENCES dim_region(region_key)
    FOREIGN KEY (date_key) REFERENCES dim_time(date_key)
)
""")

print("✓ fact_value table created successfully.")

## Close Connection

In [ ]:
conn.close()
print("✓ Connection closed. Data Warehouse is ready for use!")